# Deep Learning Arch #2
## Embedding->CNN->LSTM->Dense

In [1]:
from keras.layers import Embedding, Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import one_hot
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import os
import csv 
import sys
import re
import scipy
import numpy as np

csv.field_size_limit(sys.maxsize)

Using TensorFlow backend.


131072

In [2]:
def find_vocab_size(trainset):
    vocab = {}
    max_length = 0
    for instance in trainset:
        tokens = instance.split()

        if len(tokens) > max_length:
            max_length = len(tokens)

        for token in tokens:
            if token in vocab:
                vocab[token] += 1
            else:
                vocab[token] = 0

    return vocab, max_length

In [3]:
callback_list = []
callback_list.append(EarlyStopping(monitor='val_loss',
                                   min_delta=0,
                                   patience=2,
                                   verbose=0,
                                   mode='auto'))

In [4]:
df_all = pd.read_pickle('../../train_data_ASTC.pkl')

domains = ['apache', 'fdroid', 'github']
for domain in domains:
    print("CURRENT DOMAIN: {}".format(domain))
    df = df_all[df_all['domain'] == domain]
    print("Dataframe shape: ", df.shape)
    
    

CURRENT DOMAIN: apache
Dataframe shape:  (46341, 3)
CURRENT DOMAIN: fdroid
Dataframe shape:  (20286, 3)
CURRENT DOMAIN: github
Dataframe shape:  (62771, 3)


In [8]:
df["source_code"].values[0]

'package isPackage ;  import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ; import isImport ;  /** * isComment */ public class isClassOrIsInterface extends isClassOrInterfaceInstantiation {  /** * isComment */ private int isVariable ;  /** * isComment */ private int isVariable ;  /** * isComment */ private float isVariable = isDoubleConstant ;  /** * isComment */ private float isVariable = isDoubleConstant ;  /** * isComment */ private float isVariable = isDoubleConstant ;  /** * isComment */ private int isVariable ;  /** * isComment */ private int isVariable = isNameExpr . isMethod (" isStringConstant ");  /** * isComment */ private int isVariable = isNameExpr . isMethod (" isStringConstant ");  /** * isComment */ private int isVariable = isNameExpr . isFieldAccessExpr ;  

In [ ]:
labels = list(set(df['target'].values))
    X = []
    Y = []

    print("Preparing lists...")
    for index, row in df.iterrows():
        X.append(row["source_code"])
        Y.append(row["target"])

    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    y_binary = to_categorical(encoded_Y)

    X_train, X_test, y_train, y_test = train_test_split(X, y_binary, train_size = 0.80, random_state=42)

    vocab, max_length = find_vocab_size(X)

    vocab_size = len(vocab) + 100
    encoded_docs_train = [one_hot(d, vocab_size) for d in X_train]
    padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')

    encoded_docs_test = [one_hot(d, vocab_size) for d in X_test]
    padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

    model = Sequential()
    # input
    model.add(Embedding(vocab_size, 128, input_length=max_length))    
    model.add(SpatialDropout1D(0.2))
    # convolutional
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(64))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(y_binary.shape[1], activation='softmax'))
    # compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    print('################# Model ###############')
    print(model.summary())

    model.fit(padded_docs_train, y_train, epochs=10, verbose=1, validation_split=0.2, callbacks=callback_list)
    # evaluate the model
    loss, accuracy = model.evaluate(padded_docs_test, y_test, verbose=1)
    print('Accuracy: %f' % (accuracy*100))